# companies 
house match
## The Problem:
_"We frequently have to map informal company names to the official company name, which may differ significantly.
Demonstrate how you would carry this out mapping a British company to the data in companies house (available at https://www.gov.uk/government/publications/a-list-of-all-companies-registered-with-companies-house-to-date). You may consider the company name, location or any other information you consider relevant."_


### Defining the problem space  

Matching an informal company name to official company name. 

__Noteworthy matching issues for company names__
1. Phonetic variations: Kohl’s versus Coles
2. Typographical mistakes: Microsoft vs. Microsft
3. Contextual differences: Company vs. Organization
4. Reordered terms: Sam Hopkins vs. Hopkins Sam
5. Prefixes and suffixes: AJO Technology Company Limited vs. AJO tech Private Co., Ltd.
6. Abbreviations, nicknames, and initials: AJ Wilson vs. Alex Jane Wilson
9. Truncated letters and missing or extra spaces: Chu Kong Transport Company vs. ChuKong Transport Co. Ltd.


### Assumptions:
- The definition of an informal company name is missing. I am assuming what is meant as an example, is when someone would say:

_I work for Barclays_ rather than _I work for Barclays plc_

- The best option is to get the companies registration number, as each company in the UK is required to have one by law and it acts as a reliable unique identifier. Job done. However for the sake of the argument we need to get a non-matching name to an offical one using only the name. 
- I'm assuming the matching would need to come from a messy dataset that we need to reconfigure. 


## Solution 1 _string tokenisation and Fuzzy matching_
This won't be perfect, however it will be able to match quite a lot of similar words in orders and return a confidence score. The steps required for this project are: 

__Step 1: Data cleaning and preparation__ 
- data cleaning - removing punctuation marks that will interfer with the probability returned by the fuzzy matching
- string tokenisation - a string becomes an object to be assessed
- identify high frequency words that are unrelated to company names - make a stop words object to cleanse off
- remove company abbreviation information (Reg|Ltd|PLC|NV|LTD|LLC|INC|LLP|US) from both lists including the companies house list this will decrease the differences between the two groups while retaining all of the original information. 

__Step 2: Fuzzy matching__
- Matrix based on fuzzy match
- Assess match and longest word algorithm to return confidence score
- Assign matched name as true value

__Step 4 Assessing all Available information:__
Combine results in a table and match remaining details

## Solution 2 _API for company matching_
Considering this is a real Business problem I'm not surprised that there is an API that does it. We can develop one and use this to test against our results. Steps for using this solution are different within the business.
https://rapidapi.com/interzoid/api/company-matching-advanced

__Issues Encountered__ 
1. I can't find a list of bad company names so I'll have to make one up. There is an obvious risk for the marker, I could make my answers look really amazing by adjusing the list until the I get a positive result. I won't. I promise. 



In [ ]:
%reset # clear defined variables

In [5]:
#Libraries
import sys
import numpy as np
import pandas as pd
import zipfile as zf
import re
import string
import sklearn
import distance
import Levenshtein
import requests

In [6]:
{sys.executable}

{'C:\\Users\\ellio\\Anaconda3\\python.exe'}

In [7]:
%matplotlib inline
from IPython.display import Image
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fastparquet import write
from fastparquet import ParquetFile
from pandas.io.json import json_normalize

## Import data
### Bad company names
1. create a list of bad names which will act as our informal names list
2. load the database from companies house

In [8]:
bad_company_names = ['Plastics //','& Sewell Secretaries' ,'Int!!','Diamond - and CO','Talent LTD','Bioga & sons', 'House LLC', 'And Haseler US', 
                     'Business Interiors PLC', 'Nutrition' ,'Associates and Parners ','Exchange ','Johnsons Accounting ','Commodities' ,'& R Catering ','Isaf ','Smiths ']


### Companies house data
Read the companies house data from the URL once at the start, comment out after you have created the parquet file. That way the run time won't be ridiculous

In [9]:
# read in the companies house data with the clean names we are matching to. The zip is available from the URL here: http://download.companieshouse.gov.uk/BasicCompanyData-2020-11-01-part1_6.zip
# df = pd.read_csv('BasicCompanyData-2020-11-01-part1_6.zip')
# write('companieshouse.parq', df)
# ok this file is huge, as this is a proof of concept I'll only load the first 500k rows to save compute. 

In [10]:
# pf = ParquetFile('companieshouse.parq')
# companies_house_names = pf.to_pandas(['CompanyName', 'RegAddress.County', ' CompanyNumber']) # bring the company number back so that you can map it.

# sys.getsizeof(companies_house_names)
# #file too big, read in 400k rows randomly

# sample_companies_house = companies_house_names.sample(n = 300)

# sample_companies_house.shape

sample_companies_house = pd.read_csv('sample_companies_house.csv')

## Utlity functions

In [16]:
# 1. clean out any special characters that will reduce the efficacy of 
def clean_special_characters(text):
    """ cleans all special characters which will reduce the accuracy of the fuzzy match.
    
    Parameters:
    -----------
    text - input of either list or dataframe of bad company names 
    
    Returns:
    --------
    dataframe of cleaned names without punctuation
    """
    
    cleaned_names_list = []
    for i in text:
        my_new_string = re.sub("[^a-zA-Z0-9\s]+", '', i) # takes out all special characters
        cleaned_names_list.append(my_new_string)
#         clean_data_frame = pd.DataFrame(cleaned_names_list, columns=['bad_company_names'])
    return cleaned_names_list        


# 2. remove stop words
def remove_stop_words(text):
    
    """ inconsistancies between things like Limited and LTD will reduce accuracy of a fuzzy match and as they aren't unique at all
     if we remove them from both then our match accuracy will increase
         
    Parameters:
    -----------
    companies: a dataframes or list of company names 
    
    Returns:
    ------------
    A data frame with the company names which:
    """
    
    
    new_df = pd.DataFrame()
    
    my_stopwords = ['Reg', 'Ltd', 'PLC', 'NV', 'LTD', 'LLC', 'INC', 'LLP', 'US']
    
    new_df['original'] = text
    
    new_df['cleaned'] = list(map(lambda x: x.lower().split(), text))
    
    new_df['filtered'] = list(map(lambda line: list(filter(lambda word: word not in my_stopwords, line)), new_df.cleaned)) # removes stopwords
    
    new_df['cleaned_words'] = list(map(lambda tokens: ' '.join(tokens), new_df.filtered)) 
    
    return new_df

# 3. 
def fuzzy_merge(df_1, df_2, key1, key2, threshold, limit):
    """
    Parameters:
    ---------------
    df_1: the bad company value dataframe
    df_2: the clean values to join on
    key1: key column of the left table
    key2: key column of the right table
    threshold: how close the matches should be to return a match, based on Levenshtein distance
    limit: the amount of matches that will get returned, these are sorted high to low
    
    Returns:
    ---------
    dataframe with left dataset and right matches
    
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    
    df_1['matches'] = m2

    return df_1


In [13]:
without_special = clean_special_characters(bad_company_names)

without_special

['Plastics ',
 ' Sewell Secretaries',
 'Int',
 'Diamond  and CO',
 'Talent LTD',
 'Bioga  sons',
 'House LLC',
 'And Haseler US',
 'Business Interiors PLC',
 'Nutrition',
 'Associates and Parners ',
 'Exchange ',
 'Johnsons Accounting ',
 'Commodities',
 ' R Catering ',
 'Isaf ',
 'Smiths ']

In [14]:
ready_to_match = remove_stop_words(without_special)

ready_to_match

,original,cleaned,filtered,cleaned_words
0,Plastics,[plastics],[plastics],plastics
1,Sewell Secretaries,"[sewell, secretaries]","[sewell, secretaries]",sewell secretaries
2,Int,[int],[int],int
3,Diamond and CO,"[diamond, and, co]","[diamond, and, co]",diamond and co
4,Talent LTD,"[talent, ltd]","[talent, ltd]",talent ltd
5,Bioga sons,"[bioga, sons]","[bioga, sons]",bioga sons
6,House LLC,"[house, llc]","[house, llc]",house llc
7,And Haseler US,"[and, haseler, us]","[and, haseler, us]",and haseler us
8,Business Interiors PLC,"[business, interiors, plc]","[business, interiors, plc]",business interiors plc
9,Nutrition,[nutrition],[nutrition],nutrition


In [15]:
fuzzy_match_to_CH = fuzzy_merge(ready_to_match, sample_companies_house, 'cleaned_words', 'CompanyName', threshold=90, limit=2)

fuzzy_match_to_CH

# mergeme.to_csv('fuzmatch.csv')

,original,cleaned,filtered,cleaned_words,matches
0,Plastics,[plastics],[plastics],plastics,"AGM PLASTICS LIMITED, APPLIED WINDOWS DOORS AN..."
1,Sewell Secretaries,"[sewell, secretaries]","[sewell, secretaries]",sewell secretaries,BISHOP & SEWELL SECRETARIES LIMITED
2,Int,[int],[int],int,CCM INTERNATIONAL LTD
3,Diamond and CO,"[diamond, and, co]","[diamond, and, co]",diamond and co,
4,Talent LTD,"[talent, ltd]","[talent, ltd]",talent ltd,
5,Bioga sons,"[bioga, sons]","[bioga, sons]",bioga sons,
6,House LLC,"[house, llc]","[house, llc]",house llc,
7,And Haseler US,"[and, haseler, us]","[and, haseler, us]",and haseler us,
8,Business Interiors PLC,"[business, interiors, plc]","[business, interiors, plc]",business interiors plc,
9,Nutrition,[nutrition],[nutrition],nutrition,A-RICH NUTRITION LTD


# Solution 2 API requests

In [ ]:
get_bad_names(bad_company_names, bad_names)

In [ ]:
url = "https://company-name-match.p.rapidapi.com/getcompanymatch"

querystring = {"company":"Close enough"}

headers = {
    'x-rapidapi-key': "xxxxxxxxxx",
    'x-rapidapi-host': "company-name-match.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)
payload = response.json()
print(payload)

## fun fact -  in case you're curious what the longest name of a registered company in the UK is:

![subtitle](img/long_name.png)

Things done:
1. remove punctuation
2. remove stopwords
3. fuzzy match

Things to be done next time:  
1. cluster companies - takes care of duplicate values
2. stemming - takes stem of words, can increase accuracy
3. write a loop from the API

The Levenshtein Distance
The Levenshtein distance is a metric to measure how apart are two sequences of words. In other words, it measures the minimum number of edits that you need to do to change a one-word sequence into the other. These edits can be insertions, deletions or substitutions. This metric was named after Vladimir Levenshtein, who originally considered it in 1965.


affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=.95)



